In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
import pandas as pd
from PIL import Image
import os
from torch.utils.data import Dataset
from nn import CustomImageDataset, FeedForwardNN, test_model, validate, CNN, train_model, set_seed, CNN_LSTM, CNNTransformer
from torch.optim.lr_scheduler import ReduceLROnPlateau


In [7]:
input_size = 28 * 28  # Flattened image size (28x28)
num_classes = 94  # Number of classes (modify if needed)
learning_rate = 0.001
num_epochs = 20
batch_size = 64
set_seed(42)

dataset = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="train")
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
dataset_test = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="test")
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
val_dataset = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="validation")
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)




In [ ]:
# Model, Loss, and Optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeedForwardNN(input_size=input_size, num_classes=num_classes, hidden_size=288).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0

for epoch in range(num_epochs):
    # Training step
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation step
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(data_loader):.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        print("Validation loss improved! Saving model...")
    else:
        counter += 1
        print(f"Validation loss did not improve. Patience: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered!")
            break
    scheduler.step(val_loss)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeedForwardNN(input_size=input_size, num_classes=num_classes, hidden_size=288).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0
train_model(num_epochs, data_loader, val_loader, device, model, criterion, optimizer, scheduler, patience)

Epoch [1/20], Train Loss: 2.7277, Validation Loss: 1.9541, Validation Accuracy: 46.16%
Validation loss improved! Saving model...
Epoch [2/20], Train Loss: 1.6439, Validation Loss: 1.6032, Validation Accuracy: 54.42%
Validation loss improved! Saving model...
Epoch [3/20], Train Loss: 1.3561, Validation Loss: 1.2871, Validation Accuracy: 63.24%
Validation loss improved! Saving model...
Epoch [4/20], Train Loss: 1.2007, Validation Loss: 1.3199, Validation Accuracy: 61.48%
Validation loss did not improve. Patience: 1/3
Epoch [5/20], Train Loss: 1.1048, Validation Loss: 1.2510, Validation Accuracy: 63.65%
Validation loss improved! Saving model...
Epoch [6/20], Train Loss: 1.0317, Validation Loss: 1.0397, Validation Accuracy: 68.83%
Validation loss improved! Saving model...
Epoch [7/20], Train Loss: 0.9513, Validation Loss: 0.9510, Validation Accuracy: 71.65%
Validation loss improved! Saving model...
Epoch [8/20], Train Loss: 0.9137, Validation Loss: 0.9678, Validation Accuracy: 70.87%
Valid

In [ ]:
torch.save(model, "FeedForward.pth")

In [9]:
dataset_test = CustomImageDataset(csv_dir="ascii_file_counts.csv", data_dir="test")
data_loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)
test_model(model, data_loader_test, device, criterion)

Test Loss: 0.7278, Test Accuracy: 76.98%


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0

for epoch in range(num_epochs):
    # Training step
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation step
    val_loss, val_accuracy = validate(model, val_loader, criterion, device)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {running_loss/len(data_loader):.4f}, "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

    # Early stopping logic
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        print("Validation loss improved! Saving model...")
    else:
        counter += 1
        print(f"Validation loss did not improve. Patience: {counter}/{patience}")
        if counter >= patience:
            print("Early stopping triggered!")
            break
    scheduler.step(val_loss)

In [ ]:
test_model(model, data_loader_test, device, criterion)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_LSTM(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0
train_model(num_epochs, data_loader, val_loader, device, model, criterion, optimizer, scheduler, patience)

In [ ]:
test_model(model, data_loader_test, device, criterion)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNTransformer(n_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()  # Suitable for classification tasks
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', factor = .2, patience =10)  # Reduce LR by 10x every 5 epochs
best_val_loss = float('inf')
patience = 3  # Number of epochs to wait for improvement
counter = 0
train_model(num_epochs, data_loader, val_loader, device, model, criterion, optimizer, scheduler, patience)

In [ ]:
test_model(model, data_loader_test, device, criterion)

In [ ]:
torch.save(model,"transformer.pth")

In [ ]:
test = torch.load("transformer.pth")
test_model(test,data_loader_test, device, criterion)

In [ ]:
g